In [1]:
import gradio as gr
import joblib
import pandas as pd
import numpy as np

In [2]:
# Load the saved model
try:
    model = joblib.load('best_random_forest_model.joblib')
except FileNotFoundError:
    print("Error: The model file 'best_random_forest_model.joblib' was not found.")
    print("Please make sure you have run the code to train and save the model.")
    exit()

In [3]:
# Define the features your model was trained on
# These should match the order and names of your columns
# We'll use a simplified set for this example
numeric_features = ["amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest", "balanceDiffOrig", "balanceDiffDest", "hour_of_day"]
categorical_features = ["type"]

In [4]:
# Define the prediction function
def predict_fraud(transaction_type, amount, oldbalanceOrg, newbalanceDest, oldbalanceDest, newbalanceOrig, step):
    
    # Create the new engineered features
    balanceDiffOrig = oldbalanceOrg - newbalanceOrig
    balanceDiffDest = newbalanceDest - oldbalanceDest
    hour_of_day = step % 24
    
    # Create a DataFrame from the inputs in the correct order
    input_data = pd.DataFrame([[
        amount, 
        oldbalanceOrg, 
        newbalanceOrig,
        oldbalanceDest,
        newbalanceDest, 
        balanceDiffOrig,
        balanceDiffDest,
        hour_of_day,
        transaction_type
    ]], columns=[*numeric_features, *categorical_features])
    
    # The ColumnTransformer inside the pipeline expects a DataFrame
    # Note: We do not include 'step' in the final data for the model, as it was used to create 'hour_of_day'
    
    # Make the prediction
    prediction = model.predict(input_data)
    
    # Return a user-friendly result
    if prediction[0] == 1:
        return "Fraudulent Transaction Detected!"
    else:
        return "Legitimate Transaction"


In [5]:
# Define the Gradio interface
# The inputs must match the order of arguments in the predict_fraud function
app = gr.Interface(
    fn=predict_fraud,
    inputs=[
        gr.Dropdown(choices=["CASH_OUT", "TRANSFER", "PAYMENT", "CASH_IN", "DEBIT"], label="Transaction Type"),
        gr.Number(label="Amount"),
        gr.Number(label="Old Balance of Sender"),
        gr.Number(label="New Balance of Sender"),        
        gr.Number(label="Old Balance of Receiver"),
        gr.Number(label="New Balance of Receiver"),

        gr.Number(label="Step (Time)", minimum=0, maximum=23),
    ],
    outputs="text",
    title="Fraud Detection App",
    description="Enter transaction details to predict if it is fraudulent."
)


In [6]:
# Launch the app
app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c541237c5f0b9ad4d3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
